In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(figsize=(6,6))

# Parameters for multiplicative clamped rate
r = 0.88            # constant multiplier (decay factor < 1)
# r = 0.9377           # constant multiplier (decay factor < 1)
steps = 100          # number of "rewards"
y0 = 0.9            # starting value
y_min, y_max = 0.0, 1.0  # clamps

# Generate clamped multiplicative curve
y_clamped = [y0]
for _ in range(1, steps):
    next_val = y_clamped[-1] * r
    next_val = min(y_max, max(y_min, next_val))
    y_clamped.append(next_val)

x = np.arange(steps)

# Plot your exponential for comparison
a = 0.9
# c = 0.1284
c = 0.0642
d = 0
y_exp = a * np.exp(-c * x) + d

ax.plot(x, y_exp, color="blue", marker="o", label="Exponential decay")
ax.plot(x, y_clamped, color="red", marker="o", label="Clamped multiplicative rate")

ax.set_xlabel('Rewards collected')
ax.set_ylabel('P(reward)')
ax.set_ylim(-0.1,1)
ax.set_xlim(-0.5,30.5)
ax.legend()
plt.show()


In [ ]:
import math 
fig, ax = plt.subplots(1, 1, figsize=(6,6))
marker = 'o'
ax1 = ax

x = np.linspace(0, 9, 10)  # Generate 100 points between 0 and 5
a = 0.6
b = math.e  # Amplitude
c = 0.1284
d = 0

# Generate x values
y = a * pow(b, -c * x) + d

ax1.plot(x, y, color=color2, marker=marker)
# ax1.text(0.15, 0.15, f'a = {a}', color=color2)

a = 0.9
y = a * pow(b, -c * x) + d
ax1.plot(x, y, color=color1, marker=marker)
# ax1.text(2, 0.75, f'a = {a}', color=color1)

a = 0.0
y = a * pow(b, -c * x) + d
ax1.plot(x, y, color=color3, marker=marker)
# ax1.text(3, 0.05, f'a = {a}', color=color3)

ax1.set_xlabel('Rewards collected')
ax1.set_ylabel('P(reward)')
# plt.hlines(0.6, 10, 0.6, color='k', linestyle='--')
ax1.set_ylim(-0.1,1)
ax1.set_xlim(-0.5,10.5)

sns.despine()
plt.tight_layout()
plot  = all_epochs.groupby(['patch_label', 'cumulative_rewards'])['reward_probability'].first().reset_index()
sns.lineplot(data=plot, x='cumulative_rewards', y='reward_probability', hue='patch_label', ax=ax1, marker='X', palette=color_dict_label)
fig.savefig(foraging_figures+'\schematic task.svg', dpi=300)

# Transform to stops keeping the same rate of reward

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters for reward based updates
r = 0.88 # value applied before
# r = 0.9377 # value for slower decaying curve
y0 = 0.9
steps = 100 # number of rewards
n_sim = 2000
y_min, y_max = 0.0, 0.9
x = np.linspace(0, steps-1, steps)  # Generate 100 points between 0 and 5

# Simulate many runs
simulations = np.zeros((n_sim, steps))
for i in range(n_sim):
    draws = np.random.randint(0, 2, size=steps)  # 0 = no reward, 1 = reward
    y_vals = [y0]
    for t in range(1, steps):
        if draws[t] == 1:
            next_val = y_vals[-1] * r
        else:
            next_val = y_vals[-1]
        next_val = min(y_max, max(y_min, next_val))
        y_vals.append(next_val)
    simulations[i] = y_vals

# Plot
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(x, simulations.T, color='gray', alpha=0.3, linewidth=0.8)

# Stops curve using multiplicative clamped rate
# r = 0.969  # slow
r = 0.94050887087  # fast
steps = 100          # number of stops
y0 = 0.9            # starting value
y_min, y_max = 0.0, 1.0  # clamps

y_clamped = [y0]
for _ in range(1, steps):
    next_val = y_clamped[-1] * r
    next_val = min(y_max, max(y_min, next_val))
    y_clamped.append(next_val)

x = np.arange(steps)

ax.plot(x, y_clamped, color='blue', marker='o', label='Rewards')
ax.plot(x, simulations.mean(axis=0), color='red', marker='s', label='Stops')

ax.set_xlabel('Stops')
ax.set_ylabel('P(reward)')
ax.set_ylim(-0.1, 1)
ax.set_xlim(-0.5, 70.5)
ax.legend()
plt.tight_layout()
plt.show()


### Based on stops

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
r = 0.88
y0 = 0.9
steps = 30
n_sim = 2000
y_min, y_max = 0.0, 1.0
update = 'rewards'
# Simulate many runs
simulations = np.zeros((n_sim, steps))
draws_all = np.zeros((n_sim, steps), dtype=int)

for i in range(n_sim):
    draws = np.random.randint(0, 2, size=steps)  # 0 = no reward, 1 = reward
    draws_all[i] = draws

    y_vals = [y0]
    for t in range(1, steps):
        if update == 'rewards':
            if draws[t] == 1:
                next_val = y_vals[-1] * r
            else:
                next_val = y_vals[-1]
        else:  # update based on stops
            next_val = y_vals[-1] * r
            
        next_val = np.clip(next_val, y_min, y_max)
        y_vals.append(next_val)

    simulations[i] = y_vals

# Average reward probability across simulations
mean_p_reward = simulations.mean(axis=0)

# Parameters for multiplicative clamped rate
r_fast = 0.94050887087
y_clamped = [y0]
for _ in range(1, steps):
    next_val = np.clip(y_clamped[-1] * r_fast, y_min, y_max)
    y_clamped.append(next_val)

x = np.arange(steps)

# Plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(x, y_clamped, color='blue', marker='o', label='Rewards (clamped)')
ax.plot(x, mean_p_reward, color='red', marker='s', label='Stops (mean)')
ax.plot(x, simulations.T, color='gray', alpha=0.3, linewidth=0.8)

ax.set_xlabel('Stops')
ax.set_ylabel('P(reward)')
ax.set_ylim(-0.1, 1)
ax.set_xlim(-0.5, 20.5)
ax.legend()
sns.despine()
plt.tight_layout()
plt.show()